In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import datetime

def fetch_jobs(skill):
    print(f"\n🔍 Searching jobs for: {skill}")
    url = f"https://www.timesjobs.com/candidate/job-search.html?searchType=personalizedSearch&from=submit&txtKeywords={skill}&txtLocation="

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')

    job_list = soup.find_all('li', class_='clearfix job-bx wht-shd-bx')
    jobs = []

    for job in job_list:
        date_tag = job.find('span', class_='sim-posted')
        posted_date = date_tag.text.strip() if date_tag else "Not Mentioned"

        if 'few' in posted_date:  # Filter for recently posted jobs
            title = job.find('h2').text.strip()
            company = job.find('h3', class_='joblist-comp-name').text.strip()
            
            # Safe location parsing
            location = "Not Available"
            ul_tag = job.find('ul', class_='top-jd-dtl clearfix')
            if ul_tag:
                li_tags = ul_tag.find_all('li')
                if len(li_tags) > 0:
                    location = li_tags[0].text.strip()

            skills_tag = job.find('span', class_='srp-skills')
            skills = skills_tag.text.strip().replace('\n', '') if skills_tag else "Not Mentioned"

            jobs.append({
                'Title': title,
                'Company': company,
                'Location': location,
                'Skills': skills,
                'Posted': posted_date
            })

    return jobs

def save_to_csv(jobs, skill):
    if not jobs:
        print("❌ No jobs found.")
        return

    df = pd.DataFrame(jobs)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M')
    filename = f"{skill}_jobs_{timestamp}.csv"
    output_dir = "output"
    os.makedirs(output_dir, exist_ok=True)
    filepath = os.path.join(output_dir, filename)
    df.to_csv(filepath, index=False)
    print(f"\n✅ Data saved to: {filepath}")

# ----------- MAIN FUNCTION -----------
def run_scraper():
    skill = input("Enter the skill to search jobs for (e.g., python, java, data science): ")
    jobs = fetch_jobs(skill)
    save_to_csv(jobs, skill)

# Run the project
run_scraper()


Enter the skill to search jobs for (e.g., python, java, data science):  Java



🔍 Searching jobs for: Java

✅ Data saved to: output\Java_jobs_20250617_1203.csv
